In [149]:
import os
from os.path import join
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import matplotlib.pyplot as plt

from PIL import Image

In [150]:
n_epoch = 30000
size_batch = 100

def get_tensor(path):
    tensor = torch.load(path)
    tensor = np.array(tensor)
    tensor = tensor.astype(np.float32)
    return tensor

def x_batch_maker(n):
    n  = list(map(get_tensor,n))
    return torch.tensor(n)

def get_class(n):
    c = str(n).split('/')[-2]
    c = int(c.split("_")[-1])
    return c

def t_batch_maker(n):
    n = list(map(get_class,n))
    return torch.tensor(n)

test_list = [[f"E:/procon/data/4x4_LearnDirection_v7/{k}/{a*size_batch + j}.pt" for j in range(size_batch) for k in range(4)] for a in range(n_epoch)]
x_batch =  map(x_batch_maker,test_list)
t_batch =  map(t_batch_maker,test_list)

In [ ]:
class directionModel(nn.Module):
    def __init__(self):
        super().__init__()
        # self.fc1 = nn.Linear(4*4*4,64)# Input Layer to Intermediate modules
        # self.fc2 = nn.Linear(64,64) #Intermediate modules to Output Layer
        # self.fca = nn.Linear(64,64)
        # self.fc3 = nn.Linear(64,32) #Intermediate modules to Output Layer
        # self.fc4 = nn.Linear(32,32) #Intermediate modules to Output Layer
        # self.fcb = nn.Linear(32,32)
        # self.fc5 = nn.Linear(32,16) #Intermediate modules to Output Layer
        # self.fc6 = nn.Linear(16,16)
        # self.fcc = nn.Linear(16,16)
        # self.fc7 = nn.Linear(16,8)
        # self.fc8 = nn.Linear(8,8)
        # self.fc9 = nn.Linear(8,4)  
        self.c0 = nn.Conv2d(in_channels=4,    # 入力は3チャネル
                            out_channels=16,  # 出力は16チャネル
                            kernel_size=3,    # カーネルサイズは3*3
                            stride=1,         # 1pix飛ばしでカーネルを移動
                            padding=1)        # 画像の外側1pixを埋める

        self.c1 = nn.Conv2d(in_channels=16,   # 入力は16チャネル
                            out_channels=32,  # 出力は32チャネル
                            kernel_size=3,    # カーネルサイズは3*3
                            stride=1,         # 1pix飛ばしでカーネルを移動
                            padding=1)        # 画像の外側1pixを埋める

        self.c2 = nn.Conv2d(in_channels=32,   # 入力は32チャネル
                            out_channels=64,  # 出力は64チャネル
                            kernel_size=3,    # カーネルサイズは3*3
                            stride=1,         # 1pix飛ばしでカーネルを移動
                            padding=1)        # 画像の外側1pixを埋める       

        self.c3 = nn.Conv2d(in_channels=64,   # 入力は32チャネル
                            out_channels=128,  # 出力は64チャネル
                            kernel_size=3,    # カーネルサイズは3*3
                            stride=1,         # 1pix飛ばしでカーネルを移動
                            padding=1)    
        
        self.c4 = nn.Conv2d(in_channels=128,   # 入力は32チャネル
                            out_channels=256,  # 出力は64チャネル
                            kernel_size=3,    # カーネルサイズは3*3
                            stride=1,         # 1pix飛ばしでカーネルを移動
                            padding=1) 

        self.bn0 = nn.BatchNorm2d(num_features=16)   # c0用のバッチ正則化
        self.bn1 = nn.BatchNorm2d(num_features=32)   # c1用のバッチ正則化
        self.bn2 = nn.BatchNorm2d(num_features=64)   # c2用のバッチ正則化
        self.bn3 = nn.BatchNorm2d(num_features=128)
        self.bn4 = nn.BatchNorm2d(num_features=256)
        self.fc = nn.Linear(256 * 4 * 4, 4) 

    def forward(self, x):#順伝播 Forward propagation
        # x = x.view(-1,4*4*4)
        # x = self.fc1(x)
        # x = self.fc2(x)
        # x = self.fca(x)
        # x = self.fc3(x)
        # x = self.fc4(x)
        # x = self.fcb(x)
        # x = self.fc5(x)
        # x = self.fc6(x)
        # x = self.fcc(x)
        # x = self.fc7(x)
        # x = self.fc8(x)
        # y = self.fc9(x)
        h = F.relu(self.bn0(self.c0(x)))
        h = F.relu(self.bn1(self.c1(h)))
        h = F.relu(self.bn2(self.c2(h)))  
        h = F.relu(self.bn3(self.c3(h))) 
        h = F.relu(self.bn4(self.c4(h))) 
        h = h.view(-1, 256 * 4 * 4)
        y = self.fc(h)     # 全結合層
        return y

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
mymodel = directionModel().to(device)
opt = optim.Adagrad(mymodel.parameters(), lr=0.07)

In [ ]:
plot_x = []
# ロスと精度を保存するリスト（訓練用・テスト用）
list_loss_train = []
list_loss_test = []
list_acc_train = []
list_acc_test = []
list_acc_all  =[]
plot_xx = []

temp_accs = []
temp_max = 0
temp_model = mymodel
i = 0
nice = 0
kk = 0
stay = [0.25]
print(len(stay))
mymodel.train()
while i < n_epoch:
    
    sum_loss = 0.
    sum_acc = 0.
    test_acc = 0.
    opt.zero_grad()
    
    x = next(x_batch).to(device)
    t = next(t_batch).to(device)
    y = mymodel(x)
    loss = F.cross_entropy(y, t)

    # 逆伝播
    
    loss.backward()

    

    # ロスと精度を蓄積
    sum_loss += loss.item()
    sum_acc += (y.max(1)[1] == t).sum().item()

    # パラメータ更新
    
    #scheduler.step()

    batch_loss = sum_loss / len(x)
    batch_acc = sum_acc / len(x)
    list_loss_train.append(batch_loss)

    

    print(f"- now {i}")
    #print("- mean loss:", mean_loss)
    print("- batch accuracy:", batch_acc) 
    print("- max accuracy:", temp_max)
    temp_accs.append(batch_acc)
    
    if i%1500==0 and i != 0:
        ave = sum(temp_accs)/1500
        temp_accs.clear()
        list_acc_train.append(ave)
        plot_x.append(i)
        # if ave < 0.2:
        #     mymodel = temp_model
        #     i = i -100 
        

    # if batch_acc > 0.8:
    #     i += 1
    # elif batch_acc > 0.85 or kk > 100:
    #     i+= 1
    #     kk = 0
    # else:
    #     kk += 1
    opt.step()
    i += 1
    
    
    
      

print("fin v5 both steped")

#print(list_acc_train)
print(f"max acc {max(list_acc_train)}")
print(f"-mean acc {sum(list_acc_train)/len(list_acc_train)}")
plt.plot(plot_x,list_acc_train)
print(nice)

In [ ]:
ev_epoch = 100
size_batch = 10

test_list = [[f"E:/procon/data/4x4_LearnDirection_v7/{k}/{size_batch*n_epoch+a*size_batch + j}.pt" for j in range(size_batch) for k in range(4)] for a in range(ev_epoch)]
ex_batch =  map(x_batch_maker,test_list)
et_batch =  map(t_batch_maker,test_list)

In [ ]:
eve_accs = []
eve_x =[]

for i in range(ev_epoch):
    sum_acc = 0.

    x_e = next(ex_batch).to(device)
    t_e = next(et_batch).to(device)
    mymodel.eval()
    y = mymodel(x_e)
    sum_acc += (y.max(1)[1] == t).sum().item()
    mean_acc = sum_acc / len(x)
    eve_accs.append(mean_acc)
    eve_x.append(i)
    print(f"- now {i}")
    print("- batch accuracy:", mean_acc) 

print(f"-mean acc :{sum(eve_accs)/len(eve_accs)}")
plt.plot(eve_x,eve_accs)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn  as sns
ys = []
ts = []
for i in range(ev_epoch):
    ex = next(ex_batch).to(device)
    et = next(et_batch).to(device)

    y = mymodel(ex)
    y = torch.argmax(y, dim=1) # 確率の最大のインデックスを取得
    ys.append(y.cpu()) 
    ts.append(et.cpu())

ys = torch.cat(ys, dim=0)
ts = torch.cat(ts, dim=0)

# confusion matrixを表示するための関数

def plot_confusion_matrix(cm,
                          classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

confmat = confusion_matrix(ys, ts)
confmat

#array([[879,  91],
#       [ 88, 950]])

classes = ['0','1','2','3','4']

plt.figure(figsize=(6, 6))
plot_confusion_matrix(confmat, classes=classes, normalize=False)